In [0]:
# Create file structure 
!sudo apt-get install tree
!mkdir -p reddit_bitcoin_data/train/dec2017
!mkdir -p reddit_bitcoin_data/train/jun2016
!mkdir -p reddit_bitcoin_data/test/dec2017
!mkdir -p reddit_bitcoin_data/test/jun2016
# !tree --filelimit 10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [0]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
np.random.seed(seed=1)

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'caramel-pager-156904' #Put your project ID here
jun2016 = pd.io.gbq.read_gbq('''
SELECT body FROM `fh-bigquery.reddit_comments.2016_06`
where subreddit = 'Bitcoin' and length(body)>100
''', project_id=project_id, verbose=False)

dec2017 = pd.io.gbq.read_gbq('''
SELECT body FROM `fh-bigquery.reddit_comments.2017_12`
where subreddit = 'Bitcoin' and length(body)>100
''', project_id=project_id, verbose=False)

Authenticated


/usr/local/lib/python3.6/dist-packages/pandas/io/gbq.py:176: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  **kwargs


In [0]:
# Check how many data we have
print(len(dec2017))
print(len(jun2016))

258095
32972


In [0]:
# Make balance dataset
bal2017 = np.random.choice(range(0,len(dec2017)),32972, replace=False)
dec2017_bal = dec2017.iloc[bal2017]
dec2017_bal.reset_index(drop=True, inplace=True)
jun2016_bal = jun2016

In [0]:
# Seperate train test dataset
all_id = np.array(range(0,len(dec2017_bal)))
train_id = np.random.choice(range(0,len(dec2017_bal)),round(len(dec2017_bal)*0.8), replace=False)
test_id = np.random.choice(np.delete(all_id,train_id), round(len(dec2017_bal)*0.2), replace=True)

In [0]:
# Write files according to the ktrain package requirment
def write_txt_files(T='train', D='dec2017', pd2write = dec2017_bal, id_ = train_id):
  folder = './reddit_bitcoin_data/%s/%s' % (T,D)
  for id in tqdm(id_):
    filename = str(id)+'.txt'
    with open(os.path.join(folder,filename),'w') as outfile:
        pd2write.iloc[id].to_string(outfile)

In [0]:
write_txt_files(T='train', D='dec2017', pd2write = dec2017_bal, id_ = train_id)
write_txt_files(T='test', D='dec2017', pd2write = dec2017_bal, id_ = test_id)
write_txt_files(T='train', D='jun2016', pd2write = jun2016_bal, id_ = train_id)
write_txt_files(T='test', D='jun2016', pd2write = jun2016_bal, id_ = test_id)

100%|██████████| 6594/6594 [00:03<00:00, 1845.11it/s]


In [0]:
!pip3 install ktrain
import ktrain
from ktrain import text

In [0]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder('./reddit_bitcoin_data/', 
                                                                      maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['dec2017', 'jun2016'])

detected encoding: utf-8
preprocessing train...
language: en


preprocessing test...
language: en


In [0]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500
done.


In [0]:
%pwd
from google.colab import drive, files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
learner.load_model('/content/gdrive/My Drive/Colab Notebooks/mymodel')

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [0]:
learner.validate(val_data=(x_test, y_test))

              precision    recall  f1-score   support

           0       0.67      0.56      0.61      4160
           1       0.62      0.72      0.67      4160

    accuracy                           0.64      8320
   macro avg       0.64      0.64      0.64      8320
weighted avg       0.64      0.64      0.64      8320



array([[2328, 1832],
       [1172, 2988]])

In [0]:
%pwd
for e in range(2):
  learner.fit(2e-5, 1)
  learner.save_model('mymodel')

Train on 52756 samples, validate on 8320 samples
Epoch 1/1
52756/52756 [==============================] - 14440s 274ms/step - loss: 0.2810 - acc: 0.8799 - val_loss: 0.9408 - val_acc: 0.6389
Train on 52756 samples, validate on 8320 samples
Epoch 1/1
46146/52756 [=========================>....] - ETA: 29:09 - loss: 0.1684 - acc: 0.9332

In [0]:
totest = """I’m not buying coins created out of thin air like Libra!!!Why don’t I create a coin and you guys can buy it off me and we’ll all just call it money.
"""
predictor.explain(totest)